In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BeitForImageClassification, BeitImageProcessor
from PIL import Image
import pandas as pd
import time

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, csv_file, feature_extractor):
        print("Loading dataset...")
        self.data = pd.read_csv(csv_file)
        self.feature_extractor = feature_extractor
        print(f"Dataset loaded with {len(self.data)} samples.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]['image_path']
        label = self.data.iloc[idx]['label']
        image = Image.open(img_path).convert('RGB')
        inputs = self.feature_extractor(images=image, return_tensors="pt")
        return inputs['pixel_values'].squeeze(0), torch.tensor(label, dtype=torch.long)

# Load pre-trained BEiT model and feature extractor
print("Loading BEiT model and feature extractor...")
model = BeitForImageClassification.from_pretrained('microsoft/beit-base-patch16-224')
feature_extractor = BeitImageProcessor.from_pretrained('microsoft/beit-base-patch16-224')
print("Model and feature extractor loaded successfully.")

# Adjust model for binary classification
print("Modifying model for binary classification...")
model.classifier = torch.nn.Linear(model.classifier.in_features, 2)  # 2 classes
print("Model adjusted.")

# Load dataset
dataset = CustomDataset('image_data.csv', feature_extractor)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
model.to(device)

# Optimizer and loss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 5
print("Starting training...")
start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for batch_idx, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        if batch_idx % 10 == 0:
            print(f"Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss.item():.4f}")
    print(f"Epoch {epoch+1} completed. Avg Loss: {epoch_loss / len(dataloader):.4f}")

total_time = time.time() - start_time
print(f"Training completed in {total_time/60:.2f} minutes.")

# Save fine-tuned model
model.save_pretrained('fine_tuned_beit_gpu')
print("Model fine-tuned and saved successfully!")


Loading BEiT model and feature extractor...


c:\Users\Aayush\miniconda3\envs\qualcomm\Lib\site-packages\transformers\utils\deprecation.py:172: UserWarning: The following named arguments are not valid for `BeitImageProcessor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)


Model and feature extractor loaded successfully.
Modifying model for binary classification...
Model adjusted.
Loading dataset...
Dataset loaded with 2594 samples.
Using device: cuda
Starting training...
Epoch 1, Batch 0, Loss: 1.1343
Epoch 1, Batch 10, Loss: 0.5768
Epoch 1, Batch 20, Loss: 0.6670
Epoch 1, Batch 30, Loss: 0.4755
Epoch 1, Batch 40, Loss: 0.7145
Epoch 1, Batch 50, Loss: 0.3393
Epoch 1, Batch 60, Loss: 0.5504
Epoch 1, Batch 70, Loss: 0.5307
Epoch 1, Batch 80, Loss: 0.3928
Epoch 1, Batch 90, Loss: 0.2280
Epoch 1, Batch 100, Loss: 0.5563
Epoch 1, Batch 110, Loss: 0.5848
Epoch 1, Batch 120, Loss: 0.3362
Epoch 1, Batch 130, Loss: 0.3691
Epoch 1, Batch 140, Loss: 0.5101
Epoch 1, Batch 150, Loss: 0.3580
Epoch 1, Batch 160, Loss: 0.5269
Epoch 1 completed. Avg Loss: 0.4927
Epoch 2, Batch 0, Loss: 0.8834
Epoch 2, Batch 10, Loss: 0.5072
Epoch 2, Batch 20, Loss: 0.3456
Epoch 2, Batch 30, Loss: 0.6787
Epoch 2, Batch 40, Loss: 0.5761
Epoch 2, Batch 50, Loss: 0.7762
Epoch 2, Batch 60, L

KeyboardInterrupt: 